# Dataset Class
  - dataset으로 부터 사용가능한 데이터를 메모리에 로딩가능
  - batch size 지정

In [1]:

import tensorflow as tf
from tensorflow.python.data import Dataset



In [2]:
# 1. from_tensor_slices() : 입력 tensor로부터 slice 생성

# 1) x, y 변수 정의
x = tf.random.normal([5,2])
y = tf.random.normal([5])

# 2) Dataset : 5개 slice
train_ds = Dataset.from_tensor_slices((x,y))
print(train_ds)

# 5개 관측치 -> 5개 slices
for train_x, train_y in train_ds:
    print("x = {}, y = {}".format(train_x.numpy(), train_y.numpy()))

<DatasetV1Adapter shapes: ((2,), ()), types: (tf.float32, tf.float32)>
x = [-0.3391899  -0.85999423], y = 0.45995643734931946
x = [-0.05774602  0.4804866 ], y = 0.8368427157402039
x = [0.9484875 1.3075742], y = 0.9203258156776428
x = [0.91037256 1.249346  ], y = -0.16611215472221375
x = [0.55195224 0.6796783 ], y = 0.007357328664511442


In [3]:
# 2. from_tensor_slices(x,y).shuffle(buffer size).batch(size)
'''
shuffle() : tensor 행단위 서플링
     - buffer size : 선택된 data size
batch() : model에 1회 공급할 dataset size
ex) 60000(mnist) -> shuffle(10000).batch(100)
    1번째 slice : 10000 data random 추출 후 서플링 -> 순서대로 100개 image 추출 -> 1회 공급 data
    2번째 slice : next 10000 data random 추출 후 서플링 -> 100개 image 추출
    ...
    6번째 slice : next 10000 data random 추출 후 서플링 -> 100개 image 추출 -> 60000data 전체 사용
'''

# 1) x, y 변수 정의 
x2 = tf.random.normal([5,2])
y2 = tf.random.normal([5])

# 2) Dataset : 5개 관측치 대상 2묶음 -> 3 slices
train_ds2 = Dataset.from_tensor_slices((x,y)).shuffle(5).batch(2)
print(train_ds2)

# 3) 3 slice -> 1 slice
for train_x, train_y in train_ds2:
    print("x = {}, y = {}".format(train_x.numpy(), train_y.numpy()))

<DatasetV1Adapter shapes: ((None, 2), (None,)), types: (tf.float32, tf.float32)>
x = [[0.55195224 0.6796783 ]
 [0.9484875  1.3075742 ]], y = [0.00735733 0.9203258 ]
x = [[-0.05774602  0.4804866 ]
 [ 0.91037256  1.249346  ]], y = [ 0.8368427  -0.16611215]
x = [[-0.3391899  -0.85999423]], y = [0.45995644]


In [8]:
# 3. keras dataset 적용
from tensorflow.keras.datasets.cifar10 import load_data

# 1) dataset load
(x_train, y_train),(x_val, y_val) = load_data()

x_train.shape
y_train.shape

# batch size = 100 image
train_ds = Dataset.from_tensor_slices((x_train,y_train)).shuffle(1000).batch(100)

cnt = 0
for img, label in train_ds:
    cnt += 1
print(50000/100)
print("slicing size = {}".format(cnt))


500.0
slicing size = 500


In [9]:
'''
문) MNIST 데이터셋을 이용하여 train_ds, val_ds 생성하기
    train_ds : shuffle = 10000, batch = 32
    val_ds : shuffle = 10000, batch = 32
'''

from tensorflow.keras.datasets.mnist import load_data

(x_train, y_train), (x_test, y_test) = load_data()

x_train.shape # (60000, 28, 28)
x_test.shape # (10000, 28, 28)

train_ds = Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)

cnt = 0
for img, label in train_ds:
    cnt += 1
# slicing size = 1875        
# 60000 = 1875 * 32    
    
val_ds = Dataset.from_tensor_slices((x_test,y_test)).shuffle(10000).batch(32)    
    
cnt = 0
for img, label in val_ds:
    cnt += 1
# slicing size = 313    
# 10000 = 32 * 313    
    
    